## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization
from transformers import BertConfig, BertTokenizer, TFBertModel


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-tf_model.h5'
VOCAB_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-vocab.txt'
model_path_list = glob.glob('/kaggle/input/142-googleq-a-train-3fold-bert-base-unc-hidden11/' + '*last_epoch.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/142-googleq-a-train-3fold-bert-base-unc-hidden11/model_fold_1_last_epoch.h5
/kaggle/input/142-googleq-a-train-3fold-bert-base-unc-hidden11/model_fold_2_last_epoch.h5
/kaggle/input/142-googleq-a-train-3fold-bert-base-unc-hidden11/model_fold_3_last_epoch.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = BertTokenizer.from_pretrained(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
bert_config = BertConfig()
bert_config.output_hidden_states=True

def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_model = TFBertModel.from_pretrained(BERT_PATH, config=bert_config)
    sequence_output, pooled_output, hidden_states = bert_model([input_word_ids, input_masks, segment_ids])
    
#     h12 = tf.reshape(hidden_states[-1][:,0],(-1,1,768))
    h11 = tf.reshape(hidden_states[-2][:,0],(-1,1,768))
#     h10 = tf.reshape(hidden_states[-3][:,0],(-1,1,768))
#     h09 = tf.reshape(hidden_states[-4][:,0],(-1,1,768))

    x = GlobalAveragePooling1D()(h11)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.955114,0.731040,0.306423,0.590017,0.437189,0.417625,0.607930,0.657192,0.362506,...,0.925082,0.929613,0.629138,0.978516,0.965302,0.851159,0.033631,0.022649,0.866613,0.925781
1,46,0.884663,0.520792,0.007090,0.788080,0.777196,0.918919,0.543948,0.472134,0.063272,...,0.721100,0.950622,0.629359,0.979834,0.979162,0.901825,0.942350,0.102631,0.062227,0.887151
2,70,0.914601,0.657965,0.025950,0.788710,0.866827,0.931646,0.565369,0.468648,0.135595,...,0.843311,0.942571,0.608799,0.973602,0.972540,0.861534,0.107819,0.052143,0.888679,0.912575
3,132,0.899262,0.490296,0.010428,0.713652,0.774008,0.906235,0.554943,0.469043,0.050835,...,0.715592,0.958589,0.699203,0.975956,0.983528,0.901964,0.777472,0.134483,0.707494,0.886772
4,200,0.937312,0.482046,0.049884,0.865704,0.636399,0.800371,0.666362,0.624697,0.088496,...,0.760869,0.889939,0.620627,0.968857,0.958308,0.833421,0.241198,0.079090,0.620229,0.887701


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.901839,0.620329,0.053114,0.750155,0.796622,0.824312,0.565783,0.500424,0.197976,...,0.813293,0.937300,0.646734,0.970182,0.972577,0.875998,0.531238,0.117898,0.501297,0.908461
std,2812.670060,0.044457,0.130959,0.096664,0.101717,0.116574,0.153249,0.051311,0.086895,0.179964,...,0.082277,0.024309,0.046840,0.010604,0.011872,0.040718,0.339661,0.062388,0.289422,0.023094
min,39.000000,0.750045,0.342884,0.003977,0.252530,0.223607,0.157376,0.458068,0.346105,0.007553,...,0.595275,0.819700,0.492022,0.928488,0.915773,0.720904,0.004425,0.004693,0.016515,0.796944
25%,2572.000000,0.874317,0.508711,0.009951,0.692099,0.761937,0.797946,0.529787,0.445184,0.065125,...,0.750008,0.926656,0.613092,0.964780,0.967722,0.853742,0.172110,0.070244,0.238134,0.896823
50%,5093.000000,0.904453,0.616212,0.016105,0.757306,0.811199,0.879966,0.556539,0.472543,0.122838,...,0.813971,0.942574,0.649458,0.972631,0.975067,0.881455,0.617255,0.120367,0.520029,0.911933
75%,7482.000000,0.937490,0.723890,0.040576,0.817780,0.868921,0.921311,0.596032,0.540277,0.273309,...,0.886736,0.953243,0.678066,0.977909,0.981066,0.904664,0.856348,0.159113,0.757352,0.924432
max,9640.000000,0.981294,0.898832,0.755404,0.969406,0.975018,0.974407,0.739101,0.783746,0.873474,...,0.959365,0.983150,0.760819,0.990112,0.992320,0.966341,0.971826,0.369635,0.987694,0.959106
